In [1]:
using ParallelTemperingMonteCarlo, StaticArrays, DelimitedFiles

Broadly we need to save:

-Params which do not change

-Configs (the BC may change?)

-Results

Beginning with the static parameters which do not change:

In [2]:
# function writeparams(savefile, params, temp)# , ensemble, pot)
#     write(savefile,"cycles: sample_rate: n_traj: n_atoms: t_i: t_f \n")
#     write(savefile, "$(params.mc_cycles) $(params.mc_sample) $(params.n_traj) $(params.n_atoms) $(first(temp.t_grid)) $(last(temp.t_grid)) \n")
# end

function writeparams(savefile,params,temp)
    headersvec = ["cycles:" "sample_rate:" "n_traj:" "n_atoms:" "t_i:" "t_f"]
    paramsvec = [params.mc_cycles params.mc_sample params.n_traj params.n_atoms first(temp.t_grid) last(temp.t_grid)]
    writedlm(savefile, [headersvec, paramsvec], ' ' )
end

writeparams (generic function with 1 method)

In [3]:
function writeensemble(savefile,ensemble::NVT)

    headersvec = ["ensemble" "n_atom_moves" "n_atom_swaps" ]
    valuesvec = ["NVT" ensemble.n_atoms ensemble.n_atom_moves ensemble.n_atom_swaps]
    writedlm(savefile, [headersvec, valuesvec], ' ' )
end
function writeensemble(savefile,ensemble::NPT)

    headersvec = ["ensemble" "n_atom_moves" "n_volume_moves" "n_atom_swaps" "pressure"]
    valuesvec = ["NPT" ensemble.n_atoms ensemble.n_atom_moves ensembles.n_volume_moves emsemble.n_atom_swaps ensemble.pressure]
    writedlm(savefile, [headersvec, valuesvec], ' ' )
end

writeensemble (generic function with 2 methods)

In [4]:
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractDimerPotential
    coeff_vec = transpose([potential.coeff[i] for i in eachindex(potential.coeff)])

    write(savefile,"ELJ_coeffs: " )
    writedlm(savefile, coeff_vec, ' ')
end
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractDimerPotentialB

    coeff_vec_a = transpose([potential.coeff_a[i] for i in eachindex(potential.coeff_a)])
    coeff_vec_b = transpose([potential.coeff_b[i] for i in eachindex(potential.coeff_b)])
    coeff_vec_c = transpose([potential.coeff_c[i] for i in eachindex(potential.coeff_c)])
    write(savefile,"ELJB \n")
    writedlm(savefile, [coeff_vec_a, coeff_vec_b, coeff_vec_c])
end
function writepotential(savefile,potential::Ptype) where Ptype <: EmbeddedAtomPotential
    write(savefile,"EAM: $(potential.n) $(potential.m) $(potential.ean) $(potential.eCam) \n")
end 
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractMachineLearningPotential
    write("Define the potential elsewhere, it's too complicated for simple I/O \n")
end

writepotential (generic function with 4 methods)

In [5]:
function save_init(potential,ensemble,params,temp)
    if ispath("./checkpoint/params.data") == true
    else
        mkpath("./checkpoint/")
        paramsfile = open("./checkpoint/params.data","w+")
        
        writeparams(paramsfile,params,temp)
        writeensemble(paramsfile,ensemble)
        writepotential(paramsfile,potential)
        
        close(paramsfile)
    end
end


save_init (generic function with 1 method)

In [6]:
n_atoms = 13
ti = 12.
tf = 16.
n_traj = 5
temp = TempGrid{n_traj}(ti,tf) 
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles
displ_atom = 0.1 # Angstrom
n_adjust = 100
max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]
mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)
c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

a=[0.0005742,-0.4032,-0.2101,-0.0595,0.0606,0.1608]
b=[-0.01336,-0.02005,-0.1051,-0.1268,-0.1405,-0.1751]
c1=[-0.1132,-1.5012,35.6955,-268.7494,729.7605,-583.4203]
potB = ELJPotentialB{6}(a,b,c1)

ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)


MoveStrategy{13, NVT}(NVT(13, 13, 0), ["atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove"])

In [7]:
save_init(potB,ensemble,mc_params,temp)

In [8]:
testfile = open("./checkpoint/testfile.data","w+")
writedlm(testfile,[n_atoms, "just checking"])

# writeparams(testfile,mc_params,temp)
# writeensemble(testfile,ensemble)
# writepotential(testfile,potB)
close(testfile)

Now the configurations, we follow the xyz format. 

In [9]:
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

#binding sphere
bc_ne13 = SphericalBC(radius=5.32*AtoBohr) 

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

start_config = Config(pos_ne13, bc_ne13)

Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[5.339202509675499, 1.7547286276557148, 0.9552945639202041], [3.8235622875401982, -4.036693324695786, 1.2586922223528678], [3.8432525502537267, -1.2169635315645375, -4.030786193502644], [1.8515104475315411, 4.369051328639683, -3.1594504692142205], [1.8196441394724319, -0.19336921007065874, 5.399104677171738], [0.6008527770958447, 5.001668105430829, 2.668542641160762], [-5.339209779512827, -1.754731061649526, -0.9552951725503948], [-0.6008496122188508, -5.001669753738395, -2.668542209586626], [-1.851508499387977, -4.369053575565342, 3.1594501498809775], [-1.8196475277773039, 0.19336884278473446, -5.399104511708638], [-3.823559204867802, 4.036697652557049, -1.2586918476896083], [-3.8432544820617607, 1.2169644600549232, 4.030785366187147], [4.394257284152319e-6, 1.4401613097476533e-6, 7.841053896189476e-7]], SphericalBC{Float64}(101.06969058367278))

In [10]:
function writeconfig(savefile , config::Config{N,BC,T} , max_displ ) where {N,BC<:SphericalBC,T}
    writedlm(savefile,[N,"$BC r2: $(config.bc.radius2) $(max_displ[1]) $(max_displ[2]) $(max_displ[3])"])
    
    for row in config.pos
        write(savefile, "1  $(row[1]) $(row[2]) $(row[3]) \n")
    end
end
function writeconfig(savefile , config::Config{N,BC,T} , max_displ ) where {N,BC<:CubicBC,T}
    writedlm(savefile,[N,"$BC box_length: $(config.bc.radius2) $(max_displ[1]) $(max_displ[2]) $(max_displ[3])" ])
    
    for row in config.pos
        write(savefile, "1  $(row[1]) $(row[2]) $(row[3]) \n")
    end
end
function writeconfig(savefile , config::Config{N,BC,T} , max_displ ) where {N,BC<:RhombicBC,T}
    writedlm(savefile,[N,"$BC box_dims: $(config.bc.box_length) $(config.bc.box_height) $(max_displ[1]) $(max_displ[2]) $(max_displ[3])"])
    
    for row in config.pos
        write(savefile, "1  $(row[1]) $(row[2]) $(row[3]) \n")
    end
end


writeconfig (generic function with 3 methods)

In [11]:
testfile = open("./checkpoint/testfile.data","w+")
writeconfig(testfile,start_config, [1. .1 .68])
close(testfile)

Now we can write individual configs, we need the overarching function. Most likely a for i in eachindex states, in this way we have a config.$i 

In [12]:
function save_checkpoint(mc_states)
    for saveindex in eachindex(mc_states)
        checkpoint_file = open("./checkpoint/config.$saveindex","w")
        writeconfig(checkpoint_file,mc_states[saveindex].config,mc_states[saveindex].max_displ)
        close(checkpoint_file)
    end
end

save_checkpoint (generic function with 1 method)

In [13]:
function histparams(results)
    resfile = open("./checkpoint/hist_info.data","w+")
    headersvec = ["e_min" "e_max" "v_min" "v_max" "Δ_E" "Δ_V" "Δ_r2"]
    infovec = [results.en_min results.en_max results.v_min results.v_max results.delta_en_hist results.delta_v_hist results.delta_r2]
    writedlm(resfile,[headersvec , infovec], ' ')
    close(resfile)
end

histparams (generic function with 1 method)

In [14]:
function checkpoint(index,mc_states,results,ensemble::NVT;rdfsave=false)
    indexfile = open("./checkpoint/index.txt","w+")
    writedlm(indexfile,index)
    close(indexfile)
    save_checkpoint(mc_states)
    histfile = open("./checkpoint/histograms.data","w+")
    writedlm(histfile,results.en_histogram)
    close(histfile)
    if rdfsave == true 
        rdffile = open("./checkpoint/rdf.data","w+")
        writedlm(rdffile,results.rdf)
    else
    end
    
end
function checkpoint(index,mc_states,results,ensemble::NPT; rdfsave=false)
    indexfile = open("./checkpoint/index.txt","w+")
    writedlm(indexfile,index)
    close(indexfile)
    save_checkpoint(mc_states)
    histfile = open("./checkpoint/histograms.data","w+")
    writedlm(histfile,results.en_histogram)
    close(histfile)
    v_file = open("./checkpoint/volume_hist.data","w+")
    writedlm(v_file,ev_histogram)
    close(v_file)
    if rdfsave == true 
        rdffile = open("./checkpoint/rdf.data","w+")
        writedlm(rdffile,results.rdf)
    else
    end
    
end

checkpoint (generic function with 2 methods)

In [15]:
states,results = ptmc_run!(mc_params,temp,start_config,pot,ensemble)

params set
equilibration complete
MC loop done.
testing revise pt2
[18.20748176693349, 22.0728370285096, 20.35669849414488, 33.796679986707055, 39.01792291348442]
done


(MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}[MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}(12.0, 26314.5821052581, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[4.896127377364926, 2.11488203932719, 0.9338742100918467], [2.790893083743673, -3.668149909033589, 1.332763013949683], [3.6354125622269713, -0.03944221178347507, -4.222563211848992], [1.355827065063513, 6.070975184045735, -2.9757024257131195], [1.5601344333624718, 0.2744195524847849, 5.0876476771226375], [-0.13803118528924463, 5.6288408892735955, 2.9090297025270697], [-5.3702042518310975, -0.4560801572543972, -1.3475100145922074], [-1.3007430153723032, -4.326874323231836, -2.5167728544419488], [-2.723091684682722, -3.879890619078203, 3.4352728846972167], [-1.7803818217347214, 2.052938875470207, -5.251106320903675], [-4.408683278819288, 5.271942174407318, -0.7842860500489537], [-3.886621304871635, 1.74497

In [16]:
save_checkpoint(states)

In [17]:
histparams(results)

In [18]:
checkpoint(1000,states,results,ensemble)

Now time to implement the read functions


In [19]:
readfile=open("./checkpoint/params.data","r+")
paramsdata = readdlm(readfile)
close(readfile)

In [20]:
paramsdata

8×6 Matrix{Any}:
     "cycles:"     "sample_rate:"    "n_traj:"       …     "t_i:"      "t_f"
 1000.0           1.0               5.0                  12.0        16.0
     "ensemble"    "n_atom_moves"    "n_atom_swaps"        ""          ""
     "NVT"       13                13                      ""          ""
     "ELJB"        ""                ""                    ""          ""
    0.0005742    -0.4032           -0.2101           …    0.0606      0.1608
   -0.01336      -0.02005          -0.1051               -0.1405     -0.1751
   -0.1132       -1.5012           35.6955              729.76     -583.42

In [21]:
paramsvec = paramsdata[1:2,:]
ensemblevec = paramsdata[3:4,:]
potinfovec = paramsdata[5:end,:]

4×6 Matrix{Any}:
   "ELJB"      ""        ""         ""        ""         ""
  0.0005742  -0.4032   -0.2101    -0.0595    0.0606     0.1608
 -0.01336    -0.02005  -0.1051    -0.1268   -0.1405    -0.1751
 -0.1132     -1.5012   35.6955  -268.749   729.76    -583.42

In [22]:
ensemblevec

2×6 Matrix{Any}:
 "ensemble"    "n_atom_moves"    "n_atom_swaps"   ""  ""  ""
 "NVT"       13                13                0    ""  ""

In [23]:
contains(ensemblevec[2,1],"NVT") == true

true

In [24]:
if contains(ensemblevec[2,1],"NVT")
    println("done")
else
    println("not done")
end

done


In [26]:
function readensemble(ensemblevec)
    if contains(ensemblevec[1],"NVT")
        return NVT(ensemblevec[2],ensemblevec[3],ensemblevec[4])
    elseif contains(ensemblevec[1],"NPT")
        return NPT(ensemblevec[2],ensemblevec[3],ensemblevec[4],ensemblevec[5],ensemblevec[6])
    end
end


readensemble (generic function with 1 method)

In [27]:
ens=readensemble(ensemblevec[2,:])

NVT(13, 13, 0)

In [28]:
potinfovec

4×6 Matrix{Any}:
   "ELJB"      ""        ""         ""        ""         ""
  0.0005742  -0.4032   -0.2101    -0.0595    0.0606     0.1608
 -0.01336    -0.02005  -0.1051    -0.1268   -0.1405    -0.1751
 -0.1132     -1.5012   35.6955  -268.749   729.76    -583.42

In [29]:
function readpotential(potinfovec)

    if potinfovec[1,1] == "ELJPotentialEven{nlength,Float64}"
        coeffs= Vector(potinfovec[1,2:end])
        return ELJPotentialEven(coeffs)
    elseif potinfovec[1,1] == "ELJPotential{nlength,Float64}"
        coeffs = Vector(potinfovec[1,2:end])
        return ELJPotential(coeffs)
    elseif potinfovec[1,1] == "EAM:"
        return EmbeddedAtomPotential(potinfovec[1,2],potinfovec[1,3],potinfovec[1,4],potinfovec[1,5])
    elseif potinfovec[1,1] == "ELJB"
        a,b,c = potinfovec[2,:],potinfovec[3,:],potinfovec[4,:]
        return ELJPotentialB(a,b,c)
    end
end

readpotential (generic function with 1 method)

In [30]:
# coeffs= Vector(potinfovec[1,2:end])
# checkpot = ELJPotential(coeffs)
evtohartree = 0.0367493
nmtobohr = 18.8973
#parameters taken from L Vocadlo etal J Chem Phys V120N6 2004
n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

eampot = EmbeddedAtomPotential(n,m,ϵ,C,a)

testing = open("./checkpoint/testfile.data","w+")
writepotential(testing,eampot)
close(testing)
testing= open("./checkpoint/testfile.data","r+")
potinfo = readdlm(testing)
close(testing)
potinfo
testingpot = EmbeddedAtomPotential(potinfo[1,2],potinfo[1,3],potinfo[1,4],potinfo[1,5])

EmbeddedAtomPotential(8.482, 4.692, 356.5912625675561, 1.4313533476465632)

In [31]:
headers , values  = paramsvec[1,:] , paramsvec[2,:] 
headers
values

6-element Vector{Any}:
 1000.0
    1.0
    5.0
   13.0
   12.0
   16.0

In [32]:
parameters = MCParams(values[1],1,values[2],values[3],values[4],100,100)
temps=TempGrid{Int(values[3])}(values[5],values[6])

TempGrid{5, Float64}([12.0, 12.894839181882503, 13.856406460551018, 14.889677745633595, 16.0], [26314.5821052581, 24488.477972394347, 22789.096593124912, 21207.64402410914, 19735.936578943576])

In [33]:
function read_params(paramsvec)
    parameters = MCParams(paramsvec[1],0,paramsvec[2],paramsvec[3],paramsvec[4],100,100)
    temps = TempGrid{Int(paramsvec[3])}(paramsvec[5],paramsvec[6])
    return parameters,temps
end


read_params (generic function with 1 method)

In [34]:
testingindex=0
for i in 1:testingindex 
    print(i)
end

testingfile=open("./checkpoint/hist_info.data","r+")
histsfile=open("./checkpoint/histograms.data","r+")
dataread=readdlm(testingfile)
histiread = readdlm(histsfile)
close(testingfile)
close(histsfile)


In [35]:
dataread

2×7 Matrix{Any}:
   "e_min"      "e_max"    "v_min"   "v_max"   "Δ_E"       "Δ_V"   "Δ_r2"
 -0.00562997  -0.0048563  0.0       0.0       7.81488e-6  0.0     0.808558

In [36]:
histiread

5×102 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  18.0  21.0  23.0   9.0  16.0  339.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     17.0  18.0  25.0  14.0  18.0  504.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     10.0  11.0  14.0  19.0  13.0  589.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     13.0  12.0  13.0   7.0  15.0  560.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0      9.0   4.0   6.0   6.0  10.0  777.0

In [37]:
histiread[1,:]==results.en_histogram[1]
testvector = []
for row in eachrow(histiread)
    push!(testvector,Vector(row))
end
testvector[1] == results.en_histogram[1]

true

In [38]:
typeof(histiread) == Matrix{Float64}

true

In [39]:
function read_init()
    readfile=open("./checkpoint/params.data","r+")
    data=readdlm(readfile)
    close(readfile)

    paramsvec=data[2,:]
    ensemblevec=data[4,:]
    potinfovec=data[5:end,:]

    mc_params , temp = read_params(paramsvec)
    ensemble = readensemble(ensemblevec)
    potential=readpotential(potinfovec)

    return mc_params , temp,  ensemble , potential
end

read_init (generic function with 1 method)

In [40]:
param,t,ens,p = read_init()

(MCParams(1000, 0, 1, 5, 13, 100, 100), TempGrid{5, Float64}([12.0, 12.894839181882503, 13.856406460551018, 14.889677745633595, 16.0], [26314.5821052581, 24488.477972394347, 22789.096593124912, 21207.64402410914, 19735.936578943576]), NVT(13, 13, 0), ELJPotentialB{6, Any}(Any[0.0005742, -0.4032, -0.2101, -0.0595, 0.0606, 0.1608], Any[-0.01336, -0.02005, -0.1051, -0.1268, -0.1405, -0.1751], Any[-0.1132, -1.5012, 35.6955, -268.7494, 729.7605, -583.4203]))

In [41]:

function setresults(histparams,histdata,histv_data,r2data)
    results=Output{Float64}(100)
    results.en_min,results.en_max=histparams[1],histparams[2]
    results.v_min,results.v_max = histparams[3],histparams[4]
    results.delta_en_hist,results.delta_v_hist,results.delta_r2=histparams[5],histparams[6],histparams[7]

    for row in eachrow(histdata)
        push!(results.en_histogram,Vector(row))
    end
    if typeof(histv_data) == Matrix{Float64}
        for row in eachrow(histv_data)
            push!(results.ev_histogram,Vector(row))
        end
    end
    if typeof(r2data) == Matrix{Float64}
        for row in eachrow(r2data)
            push!(results.rdf,Vector(row))
        end
    end
    return results
end

setresults (generic function with 1 method)

In [42]:
configfile=open("./checkpoint/config.1","r+")
confinfo = readdlm(configfile)
close(configfile)
confinfo

15×6 Matrix{Any}:
 13                          ""          ""          ""         ""   ""
   "SphericalBC{Float64}"    "r2:"    101.07        0.313843   0.1  1.0
  1                         4.89613     2.11488     0.933874    ""   ""
  1                         2.79089    -3.66815     1.33276     ""   ""
  1                         3.63541    -0.0394422  -4.22256     ""   ""
  1                         1.35583     6.07098    -2.9757      ""   ""
  1                         1.56013     0.27442     5.08765     ""   ""
  1                        -0.138031    5.62884     2.90903     ""   ""
  1                        -5.3702     -0.45608    -1.34751     ""   ""
  1                        -1.30074    -4.32687    -2.51677     ""   ""
  1                        -2.72309    -3.87989     3.43527     ""   ""
  1                        -1.78038     2.05294    -5.25111     ""   ""
  1                        -4.40868     5.27194    -0.784286    ""   ""
  1                        -3.88662     1.7449

In [43]:
N=confinfo[1,1]
confinfo[2,1] == "SphericalBC{Float64}"
boundarycondition = SphericalBC(;radius=sqrt(confinfo[2,3]))

SphericalBC{Float64}(101.06969058367278)

In [44]:
posmatrix=confinfo[3:end,2:end]
posvecs = [posmatrix[i,:] for i in 1:N ]
# testconf = Config(posvecs,boundarycondition)

13-element Vector{Vector{Any}}:
 [4.896127377364926, 2.11488203932719, 0.9338742100918467, "", ""]
 [2.790893083743673, -3.668149909033589, 1.332763013949683, "", ""]
 [3.6354125622269713, -0.03944221178347507, -4.222563211848992, "", ""]
 [1.355827065063513, 6.070975184045735, -2.9757024257131195, "", ""]
 [1.5601344333624718, 0.2744195524847849, 5.0876476771226375, "", ""]
 [-0.13803118528924463, 5.6288408892735955, 2.9090297025270697, "", ""]
 [-5.3702042518310975, -0.4560801572543972, -1.3475100145922074, "", ""]
 [-1.3007430153723032, -4.326874323231836, -2.5167728544419488, "", ""]
 [-2.723091684682722, -3.879890619078203, 3.4352728846972167, "", ""]
 [-1.7803818217347214, 2.052938875470207, -5.251106320903675, "", ""]
 [-4.408683278819288, 5.271942174407318, -0.7842860500489537, "", ""]
 [-3.886621304871635, 1.7449783721138086, 4.400840773320395, "", ""]
 [-0.2534596603777504, 0.7989329176029321, -0.056415567962122735, "", ""]

In [45]:
testingvectors = [confinfo[2+i,2:end] for i in 1:N]
testingvectors == posvecs

true

In [46]:

function read_checkpoint_config(xyzdata)
    N=xyzdata[1,1]
    if confinfo[2,1] == "SphericalBC{Float64}"
        bc = SphericalBC(;radius=sqrt(confinfo[2,3]))
        
        max_displ = [confinfo[2,4:6]]
    elseif confinfo[2,1] == "CubicBC{Float64}"
        bc = CubicBC(confinfo[2,3])

        max_displ = [confinfo[2,4:6]]
    elseif confinfo[2,1] == "RhombicBC"
        bc = RhombicBC(confinfo[2,3],confinfo[2,4])
        max_displ = [confinfo[2,5:7]]
    end
    configvectors = [xyzdata[2+i,2:4] for i in 1:N]

    config=Config(configvectors,bc)

    return config , max_displ
end

read_checkpoint_config (generic function with 1 method)

In [47]:
testingconf,testingdispl = read_checkpoint_config(confinfo)


(Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[4.896127377364926, 2.11488203932719, 0.9338742100918467], [2.790893083743673, -3.668149909033589, 1.332763013949683], [3.6354125622269713, -0.03944221178347507, -4.222563211848992], [1.355827065063513, 6.070975184045735, -2.9757024257131195], [1.5601344333624718, 0.2744195524847849, 5.0876476771226375], [-0.13803118528924463, 5.6288408892735955, 2.9090297025270697], [-5.3702042518310975, -0.4560801572543972, -1.3475100145922074], [-1.3007430153723032, -4.326874323231836, -2.5167728544419488], [-2.723091684682722, -3.879890619078203, 3.4352728846972167], [-1.7803818217347214, 2.052938875470207, -5.251106320903675], [-4.408683278819288, 5.271942174407318, -0.7842860500489537], [-3.886621304871635, 1.7449783721138086, 4.400840773320395], [-0.2534596603777504, 0.7989329176029321, -0.056415567962122735]], SphericalBC{Float64}(101.06969058367278)), Vector{Any}[[0.3138428376721002, 0.1, 1.0]])

In [48]:
typeof(ensemble) == NVT

true

In [49]:
function rebuild_states(n_traj,ensemble,temps,potential)
    histinfofile=open("./checkpoint/hist_info.data","r+")
    histsfile=open("./checkpoint/histograms.data","r+")
    dataread = readdlm(histinfofile)
    histiread=readdlm(histsfile)
    close(histinfofile)
    close(histsfile)
    if isfile("./checkpoint/rdf.data") == true
        rdffile = open("./checkpoint/rdf.data","r+")
        rdfinfo= readdlm(rdffile)
        close(rdffile)
    else
        rdfinfo = false
    end
    if typeof(ensemble) == NPT
        ev_file = open("./checkpoint/volume_hist.data","r+")
        ev_info = readdlm(ev_file)
        close(ev_file)
    else
        ev_info = false
    end
    results = setresults(dataread[2,:],histiread,ev_info,rdfinfo)
    mcstates = []
    for index in 1:n_traj
        configfile = open("./checkpoint/config.$index","r+")
        configinfo=readdlm(configfile)
        close(configfile)
        conf,maxdisp = read_checkpoint_config(configinfo)

        state = MCState(temps.t_grid[index],temps.beta_grid[index],conf,ensemble,potential;max_displ=maxdisp[1])
        push!(mcstates,state)
    end
    return mcstates,results
end


rebuild_states (generic function with 1 method)

In [50]:
testingstates,testoutput = rebuild_states(5,ensemble,temp,pot)

(Any[MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}(12.0, 26314.5821052581, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[4.896127377364926, 2.11488203932719, 0.9338742100918467], [2.790893083743673, -3.668149909033589, 1.332763013949683], [3.6354125622269713, -0.03944221178347507, -4.222563211848992], [1.355827065063513, 6.070975184045735, -2.9757024257131195], [1.5601344333624718, 0.2744195524847849, 5.0876476771226375], [-0.13803118528924463, 5.6288408892735955, 2.9090297025270697], [-5.3702042518310975, -0.4560801572543972, -1.3475100145922074], [-1.3007430153723032, -4.326874323231836, -2.5167728544419488], [-2.723091684682722, -3.879890619078203, 3.4352728846972167], [-1.7803818217347214, 2.052938875470207, -5.251106320903675], [-4.408683278819288, 5.271942174407318, -0.7842860500489537], [-3.886621304871635, 1.7449783721138086, 4.400840773320395], [-0.2534596603777504, 0.7989329176029321, -0.056415567962122735

In [51]:
testingstates[1]

MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}(12.0, 26314.5821052581, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[4.896127377364926, 2.11488203932719, 0.9338742100918467], [2.790893083743673, -3.668149909033589, 1.332763013949683], [3.6354125622269713, -0.03944221178347507, -4.222563211848992], [1.355827065063513, 6.070975184045735, -2.9757024257131195], [1.5601344333624718, 0.2744195524847849, 5.0876476771226375], [-0.13803118528924463, 5.6288408892735955, 2.9090297025270697], [-5.3702042518310975, -0.4560801572543972, -1.3475100145922074], [-1.3007430153723032, -4.326874323231836, -2.5167728544419488], [-2.723091684682722, -3.879890619078203, 3.4352728846972167], [-1.7803818217347214, 2.052938875470207, -5.251106320903675], [-4.408683278819288, 5.271942174407318, -0.7842860500489537], [-3.886621304871635, 1.7449783721138086, 4.400840773320395], [-0.2534596603777504, 0.7989329176029321, -0.056415567962122735]], S